In [1]:
import torch
import numpy as np
import pandas as pd
from sentence_transformers import CrossEncoder, SentenceTransformer, util
import json
# from underthesea import sent_tokenize
# from vinorm import TTSnorm
# from TTS.tts.configs.xtts_config import XttsConfig
# from TTS.tts.models.xtts import Xtts
# import io
# import soundfile as sf  # Thư viện xử lý âm thanh
# from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
# from pydub import AudioSegment
# from io import BytesIO
# import librosa
# from Xu_ly_text import Xu_ly_text, Xu_ly_text_de_doc
# import re
# from dotenv import load_dotenv
# import os
# from huggingface_hub import login

# load_dotenv()

device = "cuda" if torch.cuda.is_available() else "cpu"

# # Get the Hugging Face access token from environment variables
# hf_token = os.getenv("PROJECTCB1_HUGGINGFACE_ACCESS_TOKEN")

# # Log in to Hugging Face using the access token
# if hf_token:
#     login(token=hf_token)
# else:
#     print(
#         "Access token not found. Please set the HUGGINGFACE_ACCESS_TOKEN in your .env file."
#     )

eb_model_path = "./Model/vnuis_embedding_bge"
embeddings_path = "./Data/Embedding.csv"


# Hàm nội bộ
def load_embedding_model(embedding_model_path):
    embedding_model = SentenceTransformer(
        model_name_or_path=embedding_model_path, device=device,
        trust_remote_code= True
    )
    return embedding_model

# def load_reranking_model(pr_model_path):
#     pr_model = CrossEncoder(model_name=pr_model_path, device=device, trust_remote_code=True)
#     return pr_model


def load_embeddings(embeddings_path):
    text_chunks_and_embedding_df = pd.read_csv(embeddings_path)
    
    # Convert the embedding column from a JSON string to a list of floats
    text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(json.loads)
    
    # Convert to PyTorch tensor
    embeddings = torch.tensor(
        np.array(text_chunks_and_embedding_df["embedding"].tolist()),
        dtype=torch.float32,
    ).to(device)
    
    pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")
    return embeddings, pages_and_chunks


# Khai báo các mô hình

print("Loading models... ")
# Load model embedding

embedding_model = load_embedding_model(eb_model_path)

# Load reranking
# rr_model_path = "itdainb/PhoRanker"
# reranking_model = load_reranking_model(rr_model_path)

# Dowload TTS capleaf/viXTTS
# from huggingface_hub import snapshot_download

# snapshot_download(
#     repo_id="capleaf/viXTTS", repo_type="model", local_dir="Model/TTS_model"
# )

embeddings, pages_and_chunks = load_embeddings(embeddings_path)  # Load embeddings

# Load model STT nguyenvulebinh/wav2vec2-base-vietnamese-250h

# processor.save_pretrained(stt_model_path)
# model.save_pretrained(stt_model_path)
# Hàm sử dụng cho API




/home/andv/important/chatbot_vnuis/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Loading models... 


In [2]:
def retrieve_relevant_resources(query: str, n_resources_to_return: int = 3, threshold: int =0.1):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    # Use cosine similarity instead of dot score
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    
    # Get all scores and corresponding indices, then filter based on score > 0.5
    scores, indices = torch.topk(input=cosine_scores, k=n_resources_to_return)
    filtered_scores_indices = [(score.item(), index.item()) for score, index in zip(scores, indices) if score.item() > threshold]
    
    # Extract the scores and indices after filtering
    filtered_indices = [index for _, index in filtered_scores_indices]
    
    # Take top 'n_resources_to_return' from the filtered list
    # top_scores = filtered_scores[:n_resources_to_return]
    top_indices = filtered_indices[:n_resources_to_return]
    
    context_items = [pages_and_chunks[i] for i in top_indices]
    results = [item["Relevant docs"] for item in context_items]
    # ques = [item["Question"] for item in context_items]
    # pr_results = reranking_model.rank(query, results, return_documents=True, top_k=5)
    return results


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TextStreamer
import torch

device = "cuda" # the device to load the model onto
model_path = "SeaLLMs/SeaLLMs-v3-1.5B-Chat"
model = AutoModelForCausalLM.from_pretrained(
  model_path,
  torch_dtype=torch.bfloat16, 
  device_map=device,
  # load_in_4bit = True,
  attn_implementation="flash_attention_2"
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# prepare messages to model
# Các đoạn văn có liên quan: <trích xuất các đoạn văn có liên quan từ ngữ cảnh tại đây>



In [4]:
# Function to format the prompt
def prompt_formatter_root(query: str, results: list) -> str:
    context = '- "' + '"\n\n- "'.join(results) + '"'

    base_prompt = """Hãy cho bản thân không gian để suy nghĩ bằng cách trích xuất các đoạn văn có liên quan từ ngữ cảnh dưới đây trước khi trả lời câu hỏi của người dùng.
Sử dụng các đoạn ngữ cảnh sau để trả lời câu hỏi của người dùng:

{context}

Đừng trả về cách suy nghĩ của bạn ví dụ như "dựa vào như cảnh, trong ngữ cảnh cung cấp,...". Trả về trực tiếp câu trả lời
Câu hỏi của người dùng: {query}
Trả lời:"""
    prompt = base_prompt.format(context=context, query=query)
    return prompt

In [6]:
query = "Bao nhiêu điểm thì đạt B"
results = retrieve_relevant_resources(query = query)
print(prompt_formatter_root(query=query, results=results))

Hãy cho bản thân không gian để suy nghĩ bằng cách trích xuất các đoạn văn có liên quan từ ngữ cảnh dưới đây trước khi trả lời câu hỏi của người dùng.
Sử dụng các đoạn ngữ cảnh sau để trả lời câu hỏi của người dùng:

- "Cách tính điểm đánh giá bộ phận, điểm học phần theo thang điểm 10:

Điểm đánh giá bộ phận và điểm thi kết thúc học phần được chấm theo thang điểm 10 (từ 0 đến 10), có lẻ đến một chữ số thập phân.

Điểm học phần là tổng của điểm đánh giá bộ phận và điểm thi kết thúc học phần sau khi đã tính trọng số được quy định trong đề cương học phần và được làm tròn đến một chữ số thập phân, sau đó được chuyển thành điểm chữ.

a) Loại đạt:

9,0 – 10,0 tương ứng với A+
8,5 – 8,9 tương ứng với A
8,0 – 8,4 tương ứng với B+
7,0 – 7,9 tương ứng với B
6,5 – 6,9 tương ứng với C+
5,5 – 6,4 tương ứng với C
5,0 – 5,4 tương ứng với D+
4,0 – 4,9 tương ứng với D
b) Loại không đạt:

Dưới 4,0 tương ứng với F"

- "Hạng tốt nghiệp được xác định theo điểm trung bình chung tích lũy (GPA) của toàn khóa h

In [ ]:
# def ask(query:str) -> str:
# messages = [
# {"role": "system", "content": "Bạn là một trợ lí Tiếng Việt hữu ích. Hãy trả lời câu hỏi của người dùng một cách chính xác."},
# ]
messages = [
{"role": "system", "content": "Bạn là Chatbot của Trường Quốc Tế - Đại học Quốc Gia Hà Nội. Hãy trả lời câu hỏi của người dùng một cách chính xác."},
]
# query = "Đó là những dự án nào vậy?"
lastest_conversation = []
while True:
    query = input("Nhập câu hỏi: ")
    print(query)
    results = retrieve_relevant_resources(query, n_resources_to_return=2, threshold = 0.3)
    prompt = prompt_formatter_root(query, results)
    messages.append({"role": "user", "content": prompt})
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generated_ids = model.generate(model_inputs.input_ids, 
                                do_sample=True,
                                temperature=0.1,  
                                    top_k=20,  
                                    top_p=0.95,  
                                    max_new_tokens=512,
                                    repetition_penalty = 1.05,  
                                streamer=streamer)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # old_messages.append({"role": "user", "content": query})
    # old_messages.append({"role": "assistant", "content": response})
    lastest_conversation.clear()
    lastest_conversation.extend([query, response])
    messages.pop()

xin chào
Xin chào! Tôi là VNU-IS Chatbot, tôi sẽ cố gắng giúp bạn với bất kỳ câu hỏi nào bạn có. Bạn cần hỗ trợ gì ạ?
bạn có thể làm những gì ?
ISVC đã tổ chức các sự kiện như "Tết Sum vầy", "Red Day", "ECOFISH", "Tọa đàm Thay đổi lối sống", "Reply 2010", "I Can't, We Can" và "Vầng Trăng Cho Em". ISVC cũng tổ chức các hoạt động như "Cơ hội nghề nghiệp", "Ngôn ngữ Anh" và "Định hướng đối ngoại".

Về câu hỏi của bạn, tôi sẽ trả lời như sau:

- Chào bạn, tôi là VNU-IS Chatbot, tôi sẵn sàng hỗ trợ bạn với mọi nhu cầu của bạn. Bạn cần hỗ trợ gì?


In [ ]:
# import streamlit as st
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# # Initialization
# st.set_page_config(page_title="Vietnamese Chatbot", page_icon="💬")
# st.title("Vietnamese Chatbot")

# # Define initial system message
# messages = [
#     {"role": "system", "content": "Bạn là một trợ lí Tiếng Việt hữu ích. Hãy trả lời câu hỏi của người dùng một cách chính xác."},
# ]
# lastest_conversation = []

# # Streamlit UI
# def main():
#     st.write("### Hỏi và Đáp")
#     query = st.text_input("Nhập câu hỏi:")

#     if st.button("Gửi") and query:
#         st.write(f"**Bạn hỏi:** {query}")
#         response = get_response(query)
#         st.write(f"**Trợ lý:** {response}")


# def get_response(query):
#     # Update conversation context
#     global lastest_conversation, messages
#     if lastest_conversation:
#         query = rewrite_query(query=query, lastest_conversation=lastest_conversation)

#     results = retrieve_relevant_resources(query, n_resources_to_return=5, threshold=0.5)
#     prompt = prompt_formatter_root(query, results)
#     messages.append({"role": "user", "content": prompt})
    
#     # Generate the response
#     text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     model_inputs = tokenizer([text], return_tensors="pt").to(device)
#     streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
#     generated_ids = model.generate(
#         model_inputs.input_ids,
#         do_sample=True,
#         temperature=0.1,
#         top_k=40,
#         top_p=0.95,
#         max_new_tokens=1024,
#         repetition_penalty=1.05,
#         streamer=streamer
#     )
#     generated_ids = [
#         output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
#     ]
#     response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
#     # Update messages and latest conversation
#     lastest_conversation.clear()
#     lastest_conversation.extend([query, response])
#     messages.pop()
#     return response


# if __name__ == "__main__":
#     main()


/home/andv/important/chatbot_vnuis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-31 19:48:24.675 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 19:48:24.675 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 19:48:24.702 
  command:

    streamlit run /home/andv/important/chatbot_vnuis/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-10-31 19:48:24.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-31 19:48:24.703 Thread 'MainThread': missing Scrip